In [2]:
from dotenv import load_dotenv
from langgraph.graph import StateGraph
from typing import TypedDict, Annotated
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
# extra imports 
from langgraph.checkpoint.memory import InMemorySaver

In [5]:
class State(TypedDict): 
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

llm = init_chat_model('google_genai:gemini-2.0-flash')
tool = TavilySearch(max_results=2)
tools = [tool]
llm_with_tools = llm.bind_tools(tools = tools)

def chatbot(state: State): 
    return {'messages' : [llm_with_tools.invoke(state['messages'])]}

graph_builder.add_node("chatbot", chatbot)
tool_node = ToolNode(tools = tools)
graph_builder.add_node("tools", tool_node)
# add edges 
graph_builder.add_conditional_edges('chatbot', tools_condition)
graph_builder.add_edge('tools', 'chatbot')
graph_builder.set_entry_point('chatbot')

# Adding checkpoint (Memory)
memory = InMemorySaver()
graph = graph_builder.compile(checkpointer = memory)

In [6]:
initial_state: State = {
    'messages' : [
        {'role': 'system' , 'content' : 'You are a helpful assistant. Answer user queries in a clear and concise manner, limiting responses to 3–4 lines unless the user explicitly asks for a longer and more detailed explanation.'}, 
    ]
}

config = {'configurable': {'thread_id': 1}}

def stream_graph_updates(user_input: str):
    initial_state['messages'].append({'role': 'user', 'content': user_input})
    events = graph.stream(
        initial_state, 
        config, 
        stream_mode = 'values'
    )
    for event in events:
        event['messages'][-1].pretty_print()

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

================================ Human Message =================================

Hey i am aniket
================================== Ai Message ==================================

Hello Aniket, how can I help you today?
================================ Human Message =================================

can you tell me todays headlines
================================== Ai Message ==================================
Tool Calls:
  tavily_search (bf7657c2-01e7-4fa8-8534-cb2947307906)
 Call ID: bf7657c2-01e7-4fa8-8534-cb2947307906
  Args:
    query: today's headlines
    topic: news
================================= Tool Message =================================
Name: tavily_search

{"query": "today's headlines", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.cbsnews.com/newyork/video/2-new-york-city-firefighters-hurt-in-coney-island-fire/", "title": "2 New York City firefighters hurt in Coney Island fire - CBS News", "score": 0.45231876, "publishe

In [9]:
graph.get_state(config)

StateSnapshot(values={'messages': [SystemMessage(content='You are a helpful assistant. Answer user queries in a clear and concise manner, limiting responses to 3–4 lines unless the user explicitly asks for a longer and more detailed explanation.', additional_kwargs={}, response_metadata={}, id='1f05cb6b-a80f-40b5-94dc-98cb79095104'), HumanMessage(content='Hey i am aniket', additional_kwargs={}, response_metadata={}, id='8cc822f3-a28f-4e8c-b003-9e8a1f8263af'), AIMessage(content='Hello Aniket, how can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--a46ac649-fcc0-4810-8682-aded8078b9e1-0', usage_metadata={'input_tokens': 1133, 'output_tokens': 13, 'total_tokens': 1146, 'input_token_details': {'cache_read': 0}}), SystemMessage(content='You are a helpful assistant. Answer user queries in a clear and concise manner, limiting res